Turns out CDM_PUBLIC data isn't very interesting relative to SOCRATES :(
===

In [1]:
import pandas as pd
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

np.set_printoptions(suppress=True)

cdm_raw = pd.read_csv("spacetrack_public_cdm.csv")
cdm_raw['TCA'] = cdm_raw['TCA'].astype('datetime64[ns]')

cdm1 = cdm_raw[cdm_raw.SAT1_OBJECT_TYPE == "PAYLOAD"]
cdm1.columns = cdm1.columns.str.replace("SAT_1","SAT")
cdm1.columns = cdm1.columns.str.replace("SAT1","SAT")
cdm1 = cdm1.drop(columns=cdm1.columns[cdm1.columns.str.contains("SAT_2")])
cdm1 = cdm1.drop(columns=cdm1.columns[cdm1.columns.str.contains("SAT2")])

cdm2 = cdm_raw[cdm_raw.SAT2_OBJECT_TYPE == "PAYLOAD"]
cdm2.columns = cdm2.columns.str.replace("SAT_2","SAT")
cdm2.columns = cdm2.columns.str.replace("SAT2","SAT")
cdm2 = cdm2.drop(columns=cdm2.columns[cdm2.columns.str.contains("SAT_1")])
cdm2 = cdm2.drop(columns=cdm2.columns[cdm2.columns.str.contains("SAT1")])

cdm_df = pd.concat([cdm1, cdm2]).sort_values(by="CDM_ID", ascending=True)
cdm_df = cdm_df.drop_duplicates(subset=['SAT_ID'])
cdm_df = cdm_df[["SAT_ID","SAT_NAME","TCA","MIN_RNG","PC"]]

cdm_df

,SAT_ID,SAT_NAME,TCA,MIN_RNG,PC
0,7727,COSMOS 724,2020-11-27 11:46:21.950,236,0.000172
1,10489,METEOSAT 1,2020-11-25 22:15:00.479,2546,NaN
3,23710,RADARSAT,2020-11-25 18:46:52.631,262,0.001849
4,25642,RADUGA 1-4,2020-11-26 14:31:33.119,3645,NaN
10,24292,MAGION 5,2020-11-28 13:39:43.027,2698,NaN
...,...,...,...,...,...
1794,23045,COSMOS 2275 (GLONASS),2020-12-25 02:07:22.928,4883,NaN
1870,6845,COSMOS 588,2020-12-26 03:53:17.807,282,0.000229
1938,20659,GORIZONT 20,2020-12-27 07:27:55.082,3231,NaN
1938,25315,COSMOS 2350,2020-12-27 07:27:55.082,3231,NaN


In [2]:
# get all norad_ids play with (only 125)
norad_ids = cdm_df.SAT_ID.values
as_string = ",".join(map(str,norad_ids))

In [3]:
f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{as_string}/EPOCH/>now-150/orderby/EPOCH desc/limit/100000/format/csv/emptyresult/show"

'https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/7727,10489,23710,25642,24292,21087,22489,17303,17369,15698,7593,23431,28221,28368,2142,20465,10594,22182,28470,13974,20693,18214,28112,28894,2847,4166,10991,27561,28220,2834,24652,10981,15697,23323,23814,24305,25367,28898,3081,27943,23455,16181,11111,27598,1641,7003,16611,20441,25635,25731,28479,9903,11735,4132,24842,45,22236,11765,13381,18123,20024,12156,14979,27600,14154,5917,21783,9589,15057,17584,26064,31126,10676,33062,12642,506,13916,14452,24841,8026,16969,2801,11971,14172,899,20232,22186,7250,20788,25399,670,25320,16199,12309,25395,1613,10961,27664,728,16613,18421,25394,13148,27820,23175,15077,21263,22828,23751,25941,19467,309,8458,16326,1208,1272,12994,13984,16276,20040,23045,6845,20659,25315,1360/EPOCH/>now-150/orderby/EPOCH desc/limit/100000/format/csv/emptyresult/show'

In [4]:
df = pd.read_csv("cdm_emergency_reportable_tles.csv")
df['EPOCH'] = df['EPOCH'].astype('datetime64[ns]')
df = df[["NORAD_CAT_ID","INCLINATION","SEMIMAJOR_AXIS","EPOCH"]]

In [5]:
new_df = df.merge(cdm_df,how="left",left_on="NORAD_CAT_ID",right_on="SAT_ID")
new_df["delta"] = new_df.EPOCH - new_df.TCA
new_df["ddelta"] = new_df.delta / datetime.timedelta(days=1)
new_df = new_df[(new_df.delta > datetime.timedelta(days=-14)) & (new_df.delta < datetime.timedelta(days=7))]
new_df = new_df.sort_values(by="EPOCH")

In [6]:
new_df.sample(3)

,NORAD_CAT_ID,INCLINATION,SEMIMAJOR_AXIS,EPOCH,SAT_ID,SAT_NAME,TCA,MIN_RNG,PC,delta,ddelta
4567,27943,98.1289,7056.907,2020-12-09 08:10:10.510752,27943,BILSAT 1,2020-12-03 11:47:12.732,898,0.000146,5 days 20:22:57.778752,5.849280
5888,21783,82.6059,7775.423,2020-12-05 06:39:48.638592,21783,COSMOS 2169,2020-12-14 16:53:32.821,144,0.000764,-10 days +13:46:15.817592,-9.426206
6193,26064,100.1961,7149.147,2020-12-04 09:55:38.593632,26064,FALCONSAT,2020-12-15 03:52:40.029,134,0.000821,-11 days +06:02:58.564632,-10.747933


In [7]:
pstd = new_df.groupby(by="SAT_ID")[["SEMIMAJOR_AXIS","INCLINATION"]].agg([('PERCENT_STD', lambda value: 100* value.std() / value.mean())])
pstd.columns = pstd.columns.droplevel(1)
pstd = pstd.rename(columns={'SEMIMAJOR_AXIS':'SEMIMAJOR_AXIS_PERCENT_STD','INCLINATION':'INCLINATION_PERCENT_STD'})
pstd
# cdm_df = cdm_df.merge

cdm_df = cdm_df.merge(pstd,how="left",left_on="SAT_ID",right_index=True)
cdm_df = cdm_df.sort_values("SEMIMAJOR_AXIS_PERCENT_STD", ascending=False)

Single satellite charts
===

I used an iterator here so we can quickly generate charts in jupyter by running the cell following that over and over.
Reset the iterator to start over from the highest `max_prob`

In [26]:
iter_sat_ids = iter(cdm_df.SAT_ID.values)

In [27]:
satellite = next(iter_sat_ids)
plot_df = new_df[new_df.SAT_ID == satellite]

sat_name = plot_df.head(1).SAT_NAME
sapstd = cdm_df[cdm_df.SAT_ID == satellite].SEMIMAJOR_AXIS_PERCENT_STD.values[0]
ipstd = cdm_df[cdm_df.SAT_ID == satellite].INCLINATION_PERCENT_STD.values[0]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=plot_df.ddelta, y=plot_df.SEMIMAJOR_AXIS, mode='lines', name="SEMIMAJOR_AXIS"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=plot_df.ddelta, y=plot_df.INCLINATION, mode='lines', name="INCLINATION"),
    secondary_y=True,
)

fig.update_layout(
    title_text=f"Detecting collision avoidance maneuver: {sat_name.values[0]} ({satellite})  Num TLEs: {len(plot_df)}<br>SEMIMAJOR_AXIS %std: {sapstd}%  INCLINATION %std: {ipstd}%"
)

# Set x-axis title
fig.update_xaxes(title_text="Day delta with Time of Closest Approach")

# Set y-axes titles
fig.update_yaxes(title_text="SEMIMAJOR_AXIS", secondary_y=False)
fig.update_yaxes(title_text="INCLINATION", secondary_y=True)

fig.show()